# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pandas as pd
import numpy as np
import re
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\liang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\liang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\liang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [43]:
# load data from database
engine = create_engine('sqlite:///disasterresponse.db')
df = pd.read_sql_table("disasterresponse", engine)
df = df.dropna()
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# generate X, y
X = df['message']
y = df.drop(columns=['id', 'message', 'original', 'genre'])

### 2. Write a tokenization function to process your text data

In [46]:
def tokenize(text):
    # Todo: normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Todo: tokenize text
    tokens = word_tokenize(text)
    
    # Todo: lemmatize and remove stop words
    tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
#build a pipeline#
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [59]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [60]:
#train pipeline
model = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [71]:
#make a prediction
y_pred = model.predict(X_test)

In [72]:
#define a function to evaluate accuracy using sklearn.classification_report package
def accuracy_score(y_pred,y_test):
    for i, col in enumerate(y_test):
        print(col, '\t', classification_report(y_test[col], y_pred[:, i]))

#evaluate model accuracy
accuracy_score(y_pred, y_test)


related 	               precision    recall  f1-score   support

         0.0       0.41      0.07      0.12      1048
         1.0       0.65      0.95      0.77      1977
         2.0       0.30      0.07      0.12        42

    accuracy                           0.63      3067
   macro avg       0.45      0.36      0.33      3067
weighted avg       0.56      0.63      0.54      3067

request 	               precision    recall  f1-score   support

         0.0       0.65      0.94      0.77      1990
         1.0       0.44      0.08      0.14      1077

    accuracy                           0.64      3067
   macro avg       0.55      0.51      0.45      3067
weighted avg       0.58      0.64      0.55      3067

offer 	               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3063
         1.0       0.00      0.00      0.00         4

    accuracy                           1.00      3067
   macro avg       0.50      0.50      0.50   

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

               precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      2900
         1.0       0.00      0.00      0.00       167

    accuracy                           0.94      3067
   macro avg       0.47      0.50      0.49      3067
weighted avg       0.89      0.94      0.92      3067

medical_products 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      2971
         1.0       0.00      0.00      0.00        96

    accuracy                           0.97      3067
   macro avg       0.48      0.50      0.49      3067
weighted avg       0.94      0.97      0.95      3067

search_and_rescue 	               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3011
         1.0       0.00      0.00      0.00        56

    accuracy                           0.98      3067
   macro avg       0.49      0.50      0.50      3067
weighted avg       0.96      0.98 

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

water 	               precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      2842
         1.0       0.20      0.00      0.01       225

    accuracy                           0.93      3067
   macro avg       0.56      0.50      0.49      3067
weighted avg       0.87      0.93      0.89      3067

food 	               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92      2620
         1.0       0.09      0.00      0.00       447

    accuracy                           0.85      3067
   macro avg       0.47      0.50      0.46      3067
weighted avg       0.74      0.85      0.79      3067

shelter 	               precision    recall  f1-score   support

         0.0       0.90      1.00      0.94      2750
         1.0       0.00      0.00      0.00       317

    accuracy                           0.89      3067
   macro avg       0.45      0.50      0.47      3067
weighted avg       0.80      0.89      0.85      

...

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

shops 	               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3062
         1.0       0.00      0.00      0.00         5

    accuracy                           1.00      3067
   macro avg       0.50      0.50      0.50      3067
weighted avg       1.00      1.00      1.00      3067

aid_centers 	               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      3045
         1.0       0.00      0.00      0.00        22

    accuracy                           0.99      3067
   macro avg       0.50      0.50      0.50      3067
weighted avg       0.99      0.99      0.99      3067



C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


other_infrastructure 	               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3014
         1.0       1.00      0.02      0.04        53

    accuracy                           0.98      3067
   macro avg       0.99      0.51      0.51      3067
weighted avg       0.98      0.98      0.97      3067

weather_related 	               precision    recall  f1-score   support

         0.0       0.85      0.98      0.91      2595
         1.0       0.22      0.03      0.05       472

    accuracy                           0.84      3067
   macro avg       0.54      0.51      0.48      3067
weighted avg       0.75      0.84      0.78      3067

floods 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      2986
         1.0       0.25      0.01      0.02        81

    accuracy                           0.97      3067
   macro avg       0.61      0.51      0.50      3067
weighted avg       0.95 

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
#create grid search obeject to tuning parameters
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [100, 200],
        'clf__estimator__max_features': ['sqrt', 'log2']
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [15]:
#train the grid search
cv.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000113560D5280>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_features': ['sqrt', 'log2'],
                         'clf__estimator__n_estimators': [100, 200],
                         'vect__ngram_range': ((1, 1), (1, 2))})

In [17]:
#find the best performance parameters
cv.best_params_

{'clf__estimator__max_features': 'log2',
 'clf__estimator__n_estimators': 200,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [68]:
#use the best parameters to retest the pipeline
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, ngram_range = (1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 200, max_features = 'log2')))
    ])
    

In [69]:
#train the model and make predictions
model1 = pipeline.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

In [70]:
#evaluate model accuracy
accuracy_score(y_pred1, y_test)

related 	               precision    recall  f1-score   support

         0.0       0.35      0.04      0.07      1048
         1.0       0.64      0.96      0.77      1977
         2.0       0.40      0.05      0.09        42

    accuracy                           0.63      3067
   macro avg       0.47      0.35      0.31      3067
weighted avg       0.54      0.63      0.52      3067

request 	               precision    recall  f1-score   support

         0.0       0.65      0.96      0.78      1990
         1.0       0.44      0.05      0.10      1077

    accuracy                           0.64      3067
   macro avg       0.54      0.51      0.44      3067
weighted avg       0.58      0.64      0.54      3067

offer 	               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3063
         1.0       0.00      0.00      0.00         4

    accuracy                           1.00      3067
   macro avg       0.50      0.50      0.50   

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

search_and_rescue 	               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3011
         1.0       0.00      0.00      0.00        56

    accuracy                           0.98      3067
   macro avg       0.49      0.50      0.50      3067
weighted avg       0.96      0.98      0.97      3067

security 	               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      3027
         1.0       0.00      0.00      0.00        40

    accuracy                           0.99      3067
   macro avg       0.49      0.50      0.50      3067
weighted avg       0.97      0.99      0.98      3067

military 	               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3058
         1.0       0.00      0.00      0.00         9

    accuracy                           1.00      3067
   macro avg       0.50      0.50      0.50      3067
weighted avg       0.99      1.0

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

food 	               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92      2620
         1.0       0.00      0.00      0.00       447

    accuracy                           0.85      3067
   macro avg       0.43      0.50      0.46      3067
weighted avg       0.73      0.85      0.79      3067

shelter 	               precision    recall  f1-score   support

         0.0       0.90      1.00      0.94      2750
         1.0       0.14      0.00      0.01       317

    accuracy                           0.90      3067
   macro avg       0.52      0.50      0.48      3067
weighted avg       0.82      0.90      0.85      3067

clothing 	               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      3043
         1.0       0.00      0.00      0.00        24

    accuracy                           0.99      3067
   macro avg       0.50      0.50      0.50      3067
weighted avg       0.98      0.99      0.99   

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

death 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      2982
         1.0       0.00      0.00      0.00        85

    accuracy                           0.97      3067
   macro avg       0.49      0.50      0.49      3067
weighted avg       0.95      0.97      0.96      3067

other_aid 	               precision    recall  f1-score   support

         0.0       0.86      1.00      0.93      2651
         1.0       0.22      0.00      0.01       416

    accuracy                           0.86      3067
   macro avg       0.54      0.50      0.47      3067
weighted avg       0.78      0.86      0.80      3067

infrastructure_related 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      2979
         1.0       1.00      0.01      0.02        88

    accuracy                           0.97      3067
   macro avg       0.99      0.51      0.50      3067
weighted avg       0.97      

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

weather_related 	               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92      2595
         1.0       0.31      0.01      0.02       472

    accuracy                           0.84      3067
   macro avg       0.58      0.50      0.47      3067
weighted avg       0.76      0.84      0.78      3067

floods 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      2986
         1.0       0.67      0.02      0.05        81

    accuracy                           0.97      3067
   macro avg       0.82      0.51      0.52      3067
weighted avg       0.97      0.97      0.96      3067

storm 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      2975
         1.0       0.50      0.02      0.04        92

    accuracy                           0.97      3067
   macro avg       0.74      0.51      0.51      3067
weighted avg       0.96      0.97      

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [61]:
#build a pipeline with decision tree classfication model
pipeline2 = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

In [62]:
#train the model and make predictions
model2 =pipeline2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

In [63]:
#evaluate model accuracy
accuracy_score(y_pred2, y_test)

related 	               precision    recall  f1-score   support

         0.0       0.38      0.34      0.36      1048
         1.0       0.66      0.70      0.68      1977
         2.0       0.13      0.10      0.11        42

    accuracy                           0.57      3067
   macro avg       0.39      0.38      0.38      3067
weighted avg       0.56      0.57      0.56      3067

request 	               precision    recall  f1-score   support

         0.0       0.67      0.68      0.67      1990
         1.0       0.38      0.36      0.37      1077

    accuracy                           0.57      3067
   macro avg       0.53      0.52      0.52      3067
weighted avg       0.57      0.57      0.57      3067

offer 	               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3063
         1.0       0.00      0.00      0.00         4

    accuracy                           1.00      3067
   macro avg       0.50      0.50      0.50   

In [64]:
#create grid search obeject to tuning parameters
parameters2 = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__criterion': ['gini', 'entropy'],
        'clf__estimator__max_depth': [None, 3]
    }

cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

In [65]:
#train grid search model
cv2.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001929E7D1280>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=DecisionTreeClassifier()))]),
             param_grid={'clf__estimator__criterion': ['gini', 'entropy'],
                         'clf__estimator__max_depth': [None, 3],
                         'tfidf__use_idf': (True, False)})

In [66]:
#find best performance parameters
cv2.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': 3,
 'tfidf__use_idf': False}

In [67]:
#use the best performance parameters to retest the model
pipeline2 = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultiOutputClassifier(DecisionTreeClassifier(criterion='gini', max_depth=3)))
    ])
#train the model and make predictions     
model2 = pipeline2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
#evaluate model performance
accuracy_score(y_pred2, y_test)

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

related 	               precision    recall  f1-score   support

         0.0       0.40      0.00      0.00      1048
         1.0       0.64      1.00      0.78      1977
         2.0       0.00      0.00      0.00        42

    accuracy                           0.64      3067
   macro avg       0.35      0.33      0.26      3067
weighted avg       0.55      0.64      0.51      3067

request 	               precision    recall  f1-score   support

         0.0       0.65      0.98      0.78      1990
         1.0       0.50      0.04      0.07      1077

    accuracy                           0.65      3067
   macro avg       0.58      0.51      0.43      3067
weighted avg       0.60      0.65      0.53      3067

offer 	               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3063
         1.0       0.00      0.00      0.00         4

    accuracy                           1.00      3067
   macro avg       0.50      0.50      0.50   

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

refugees 	               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3019
         1.0       0.00      0.00      0.00        48

    accuracy                           0.98      3067
   macro avg       0.49      0.50      0.50      3067
weighted avg       0.97      0.98      0.98      3067

death 	               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      2982
         1.0       0.00      0.00      0.00        85

    accuracy                           0.97      3067
   macro avg       0.49      0.50      0.49      3067
weighted avg       0.95      0.97      0.96      3067

other_aid 	               precision    recall  f1-score   support

         0.0       0.86      1.00      0.93      2651
         1.0       0.00      0.00      0.00       416

    accuracy                           0.86      3067
   macro avg       0.43      0.50      0.46      3067
weighted avg       0.75      0.86      0.80

C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liang\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### 9. Export your model as a pickle file

In [75]:
# Pickle best model
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.